In [5]:
import osmnx as ox
import networkx as nx
import folium
import pandas as pd
import numpy as np
 
from collections import defaultdict

# 상수 정의
MAX_DISTANCE = 0.5  # km
DEFAULT_COLOR = 'blue'


# 데이터 로딩 함수
def load_data(file_name):
    try:
        return pd.read_csv(file_name)
    except FileNotFoundError:
        print(f"Error: {file_name} not found.")
        return pd.DataFrame()


# 마커 추가 함수
def add_marker(m, location, popup, color, icon):
    folium.Marker(
        location=location,
        popup=popup,
        icon=folium.Icon(color=color, icon=icon, prefix='fa')
    ).add_to(m)


# 경로 계산 및 추가 함수
def add_route(m, G, start_node, end_node, color, weight=2.5, opacity=1, dash_array=None):
    route = nx.shortest_path(G, start_node, end_node, weight='length')
    route_coords = [(G.nodes[node]['y'], G.nodes[node]['x']) for node in route]
    folium.PolyLine(route_coords, color=color, weight=weight, opacity=opacity, dash_array=dash_array).add_to(m)


# 거리 계산 함수 (벡터화된 버전)
def calc_distances(lat1, lon1, lats, lons):
    R = 6371  # 지구의 반지름 (km)
    dlat = np.radians(lats - lat1)
    dlon = np.radians(lons - lon1)
    a = np.sin(dlat / 2) ** 2 + np.cos(np.radians(lat1)) * np.cos(np.radians(lats)) * np.sin(dlon / 2) ** 2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    return R * c


# 데이터 로딩
houses_df = pd.DataFrame({
    'house_id': [1],
    'latitude': [29.98497867],
    'longitude': [-90.11030819],
    'address': ["2529B Aubry St, New Orleans, LA 70119"]
})
bus_stops_df = load_data('bus_stops.csv')
schools_df = load_data('schools.csv')

# 선택된 집
selected_house = houses_df.iloc[0]
house_location = (selected_house['latitude'], selected_house['longitude'])

# 거리 계산 (벡터화 연산)
house_lat, house_lon = house_location
bus_stops_df['distance_to_house'] = calc_distances(house_lat, house_lon,
                                                   bus_stops_df['latitude'],
                                                   bus_stops_df['longitude'])

# 필터링
nearby_stops = bus_stops_df[bus_stops_df['distance_to_house'] <= MAX_DISTANCE]
filtered_paths = set(nearby_stops['path_id'])
filtered_bus_stops_df = bus_stops_df[bus_stops_df['path_id'].isin(filtered_paths)]

# 지도 생성
m = folium.Map(location=(29.9511, -90.0715), zoom_start=13)

# 집 위치 마커 추가
add_marker(m, house_location, f"Selected House: {selected_house['address']}", 'red', 'house')

# 도로 네트워크 그래프 로드
G = ox.graph_from_place("New Orleans, Louisiana, USA", network_type='walk')
nodes, _ = ox.graph_to_gdfs(G)
node_coords = dict(zip(nodes.index, nodes[['y', 'x']].values))

# 색상 매핑
path_colors = defaultdict(lambda: DEFAULT_COLOR, {1: 'blue', 2: 'green', 3: 'purple'})

# 경로별 버스 정류장 그룹화 및 경로 추가
for path_id, group in filtered_bus_stops_df.groupby('path_id'):
    path_color = path_colors[path_id]
    path_nodes = [ox.nearest_nodes(G, lon, lat) for lat, lon in zip(group['latitude'], group['longitude'])]

    for nearest_node in path_nodes:
        if nearest_node in node_coords:
            add_marker(m, node_coords[nearest_node], f"Bus Stop Path {path_id}", path_color, 'bus')

    for i in range(len(path_nodes) - 1):
        add_route(m, G, path_nodes[i], path_nodes[i + 1], path_color)

    # 학교까지의 경로 추가
    last_bus_stop = group.iloc[-1]
    school = schools_df[schools_df['school_id'] == last_bus_stop['school_id']].iloc[0]
    add_marker(m, (school['latitude'], school['longitude']), f"School: {school['name']}", 'black', 'school')

    last_node = ox.nearest_nodes(G, last_bus_stop['longitude'], last_bus_stop['latitude'])
    school_node = ox.nearest_nodes(G, school['longitude'], school['latitude'])
    add_route(m, G, last_node, school_node, path_color, dash_array='5, 5')

m